In [1]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf

with open('episodes\\book.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

#print(vocab)
#print(vocab_to_int)
#print(int_to_vocab)

#encoded contains the entire text, encoded character-wise. Example: MONICA: 29 56 ...etc where 29 is M and 56 is O
#print(encoded)

def get_batches(arr, batch_size, n_steps):
    # Get the number of characters per batch and number of batches we can make
    chars_per_batch = batch_size * n_steps
    n_batches = len(arr)//chars_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * chars_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y_temp = arr[:, n+1:n+n_steps+1]
        
        # For the very last batch, y will be one character short at the end of 
        # the sequences which breaks things. To get around this, I'll make an 
        # array of the appropriate size first, of all zeros, then add the targets.
        # This will introduce a small artifact in the last batch, but it won't matter.
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        
        yield x, y


#batches = get_batches(encoded, 10, 50)
#x,y = next(batches)

#print(x,y)

C:\Users\mayan\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mayan\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mayan\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mayan\AppData\Roaming\Python\Python37\site-

In [2]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout'''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [3]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers'''
    
    #Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [4]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [5]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [6]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [7]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        
        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [ ]:
epochs = 100
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

batch_size = 128
num_steps = 50
lstm_size = 128
num_layers =2 
learning_rate =0.001

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.6,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future majo

Epoch: 5/100...  Training Step: 1400...  Training loss: 2.2070...  0.1000 sec/batch
Epoch: 5/100...  Training Step: 1450...  Training loss: 2.1696...  0.0670 sec/batch
Epoch: 6/100...  Training Step: 1500...  Training loss: 2.1521...  0.1330 sec/batch
Epoch: 6/100...  Training Step: 1550...  Training loss: 2.1733...  0.1340 sec/batch
Epoch: 6/100...  Training Step: 1600...  Training loss: 2.1071...  0.1000 sec/batch
Epoch: 6/100...  Training Step: 1650...  Training loss: 2.1230...  0.1000 sec/batch
Epoch: 6/100...  Training Step: 1700...  Training loss: 2.1198...  0.1000 sec/batch
Epoch: 6/100...  Training Step: 1750...  Training loss: 2.1302...  0.0990 sec/batch
Epoch: 7/100...  Training Step: 1800...  Training loss: 2.0862...  0.1010 sec/batch
Epoch: 7/100...  Training Step: 1850...  Training loss: 2.1384...  0.0990 sec/batch
Epoch: 7/100...  Training Step: 1900...  Training loss: 2.0717...  0.1000 sec/batch
Epoch: 7/100...  Training Step: 1950...  Training loss: 2.0630...  0.1590 se

Epoch: 22/100...  Training Step: 6250...  Training loss: 1.7125...  0.1000 sec/batch
Epoch: 22/100...  Training Step: 6300...  Training loss: 1.7848...  0.0650 sec/batch
Epoch: 22/100...  Training Step: 6350...  Training loss: 1.7661...  0.0680 sec/batch
Epoch: 22/100...  Training Step: 6400...  Training loss: 1.7362...  0.0990 sec/batch
Epoch: 22/100...  Training Step: 6450...  Training loss: 1.7611...  0.0680 sec/batch
Epoch: 23/100...  Training Step: 6500...  Training loss: 1.7353...  0.0680 sec/batch
Epoch: 23/100...  Training Step: 6550...  Training loss: 1.7652...  0.0990 sec/batch
Epoch: 23/100...  Training Step: 6600...  Training loss: 1.7475...  0.0640 sec/batch
Epoch: 23/100...  Training Step: 6650...  Training loss: 1.7089...  0.1000 sec/batch
Epoch: 23/100...  Training Step: 6700...  Training loss: 1.7114...  0.1000 sec/batch
Epoch: 23/100...  Training Step: 6750...  Training loss: 1.7547...  0.1000 sec/batch
Epoch: 24/100...  Training Step: 6800...  Training loss: 1.7312..

Epoch: 38/100...  Training Step: 11100...  Training loss: 1.6404...  0.0630 sec/batch
Epoch: 38/100...  Training Step: 11150...  Training loss: 1.6306...  0.0640 sec/batch
Epoch: 38/100...  Training Step: 11200...  Training loss: 1.6375...  0.0990 sec/batch
Epoch: 39/100...  Training Step: 11250...  Training loss: 1.6140...  0.1000 sec/batch
Epoch: 39/100...  Training Step: 11300...  Training loss: 1.6231...  0.1000 sec/batch
Epoch: 39/100...  Training Step: 11350...  Training loss: 1.6297...  0.0650 sec/batch
Epoch: 39/100...  Training Step: 11400...  Training loss: 1.6242...  0.0650 sec/batch
Epoch: 39/100...  Training Step: 11450...  Training loss: 1.5800...  0.1000 sec/batch
Epoch: 39/100...  Training Step: 11500...  Training loss: 1.6097...  0.0640 sec/batch
Epoch: 40/100...  Training Step: 11550...  Training loss: 1.6058...  0.0990 sec/batch
Epoch: 40/100...  Training Step: 11600...  Training loss: 1.6293...  0.0990 sec/batch
Epoch: 40/100...  Training Step: 11650...  Training lo

Epoch: 54/100...  Training Step: 15900...  Training loss: 1.5999...  0.1010 sec/batch
Epoch: 55/100...  Training Step: 15950...  Training loss: 1.5578...  0.0990 sec/batch
Epoch: 55/100...  Training Step: 16000...  Training loss: 1.5634...  0.1010 sec/batch
Epoch: 55/100...  Training Step: 16050...  Training loss: 1.5474...  0.0990 sec/batch
Epoch: 55/100...  Training Step: 16100...  Training loss: 1.5789...  0.1390 sec/batch
Epoch: 55/100...  Training Step: 16150...  Training loss: 1.5679...  0.1380 sec/batch
Epoch: 55/100...  Training Step: 16200...  Training loss: 1.5480...  0.1360 sec/batch
Epoch: 56/100...  Training Step: 16250...  Training loss: 1.5753...  0.1000 sec/batch
Epoch: 56/100...  Training Step: 16300...  Training loss: 1.6000...  0.0990 sec/batch
Epoch: 56/100...  Training Step: 16350...  Training loss: 1.5252...  0.0620 sec/batch
Epoch: 56/100...  Training Step: 16400...  Training loss: 1.5776...  0.1020 sec/batch
Epoch: 56/100...  Training Step: 16450...  Training lo

In [ ]:
tf.train.get_checkpoint_state('checkpoints')

In [ ]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [ ]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [ ]:
tf.train.latest_checkpoint('checkpoints')

In [ ]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 10000, lstm_size, len(vocab), prime="Carter")
print(samp)